In [1]:
# Libraries
import json, pandas as pd, pydeck as pdk, ee, ipyfilechooser, ipywidgets, utils, datetime, concurrent.futures

In [2]:
# Initializes the Google Earth Engine APIs
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


# Exloratory analysis

The project aim is to create a Machine Learning model capable of detecting the dates when a crop field has been manured, using satellite data. <br>
Before even starting considering models, it is useful to perform exploratory analysis on the obtained crop fields dataset.

## Import a JSON file containing crop fields details

In [3]:
# Choose the file (it must be a JSON file)
file_chooser = ipyfilechooser.FileChooser(path='../Datasets/main/', filename="main-crops.json", select_default=True, use_dir_icons=True, filter_pattern='*.json')
display(file_chooser)

FileChooser(path='/Users/francesco/Documents/Documents/Università (Università degli Studi, PV)/2_ MAGISTRALE…

In [4]:
# Load JSON data from file
with open(file_chooser.selected) as f:
    data = json.load(f)

# Create DataFrame with properties
fields_df = pd.DataFrame([f["properties"] for f in data["features"]])

# Add column with coordinates for each field
fields_df["polygon_coordinates"] = [[tuple(c) for c in p] for f in data["features"] for p in f["geometry"]["coordinates"]]

In [5]:
# Show the dataframe
fields_df

,crop_field_name,manure_dates,polygon_coordinates
0,P-BLD,[2022-05-26],"[(-4.202723286616649, 43.39683579015289), (-4...."
1,P-BLLT1,[2022-05-16],"[(-4.085622203603083, 43.429605845026266), (-4..."
2,P-BLLT2,[2022-05-26],"[(-4.084840437376829, 43.430826294936246), (-4..."
3,P-Cardana,[2022-02-24],"[(8.658803437240303, 45.85842753378426), (8.65..."
4,P-CBRCS1,[2022-05-26],"[(-4.200826431306206, 43.39067464298489), (-4...."
5,P-CBRCS2,[2022-05-26],"[(-4.204911872695676, 43.3876170244562), (-4.2..."
6,P-CLGT,[2022-05-16],"[(-4.111699726693341, 43.39830644556494), (-4...."
7,P-CLMBRS,[2022-05-26],"[(-4.544769098140127, 43.38040395682432), (-4...."
8,P-CMNTR,[2022-05-16],"[(-4.147208715069137, 43.40038457218137), (-4...."
9,P-DR,[2022-03-21],"[(-4.142486752802821, 43.396858931472195), (-4..."


## Show crop fields on a map

In [6]:
# Define the layer with a tooltip
layer = pdk.Layer(
    "PolygonLayer",
    data=fields_df,
    get_polygon="polygon_coordinates",
    get_fill_color=[255, 255, 0, 100],
    get_line_color=[255, 255, 0, 100],
    stroked=True,
    filled=True,
    lineWidthMinPixels=3,
    pickable=True,
    auto_highlight=True,
)

# Define the initial view state of the map
view_state = pdk.ViewState(
    longitude=fields_df.polygon_coordinates[0][0][0],
    latitude=fields_df.polygon_coordinates[0][0][1],
    zoom=7.8
)

# Create the map with the layers and the initial view state
r = pdk.Deck(layers=layer, initial_view_state=view_state,)

# Show the map
r.show()


DeckGLWidget(carto_key=None, custom_libraries=[], google_maps_key=None, json_input='{\n  "initialViewState": {…

It can be noticed that our fields are placed in the Northern part of Spain. Please consider generalization issue.

## Features extraction

The objective is to generate a dataset that contains for each field, for each time the satellites has passed on the field (in a period, specified by the user), all the phisical indicators that will be further used to build the final model. <br>
This procedure has been designed to be performed in parallel in order to exploit the computational power of the machine (since each field is indipendent with the others).

In [7]:
start_date_widget = ipywidgets.widgets.DatePicker(description='Start date', value=datetime.date(2022, 1, 1), disabled=False)
display(start_date_widget)

end_date_widget = ipywidgets.widgets.DatePicker(description='End date', value=datetime.date(2022, 12, 31), disabled=False)
display(end_date_widget)

DatePicker(value=datetime.date(2022, 1, 1), description='Start date')

DatePicker(value=datetime.date(2022, 12, 31), description='End date')

In [9]:
def process_field(field):
    # Get the field name, manure dates, and polygon coordinates
    field_name = field['crop_field_name']
    manure_dates = field['manure_dates']
    polygon = ee.Geometry.Polygon(field['polygon_coordinates'])
    
    # Filter Sentinel 2 collection
    s2_collection = ee.ImageCollection('COPERNICUS/S2_SR')
    s2_filtered = s2_collection.filterBounds(polygon).filterDate(str(start_date_widget.value), str(end_date_widget.value)) \
                                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))

    ''' Filter Sentinel 1 collection for dates that match the Sentinel 2 collection
    There is an issue since Sentinel 1 and Sentinel 2 can pass over the same geographical
    area on completely different days, as they have different orbits...

    s1_collection = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
    s1_filtered = s1_collection.filterBounds(polygon).filterDate(start_date_widget.value, end_date_widget.value) \
                                  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
                                  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
                                  .filter(ee.Filter.eq('instrumentMode', 'IW')) \
                                  .filterMetadata('resolution_meters', 'equals', 10) \
                                  .filter(ee.Filter.equals(leftField='system:time_start', rightField='system:time_start', **{'join': ee.Filter.equals()}))
    '''

    # Get distinct dates from the Sentinel 2 collection and put into the date_range list
    s2_dates = s2_filtered.aggregate_array('system:time_start').map(lambda time_start: ee.Date(time_start).format('YYYY-MM-dd'))
    s2_dates_distinct = s2_dates.distinct().sort()
    date_range = pd.to_datetime(s2_dates_distinct.getInfo())
    
    # Create a list to store rows for the field
    acquisitions = []
    for date in date_range:
        # Calculate indices for the date
        ndvi = utils.calculate_ndvi(s2_filtered, date, polygon)
        eomi1 = utils.calculate_eomi(s2_filtered, date, polygon, 1)
        eomi2 = utils.calculate_eomi(s2_filtered, date, polygon, 2)
        eomi3 = utils.calculate_eomi(s2_filtered, date, polygon, 3)
        eomi4 = utils.calculate_eomi(s2_filtered, date, polygon, 4)
        nbr2 = utils.calculate_nbr2(s2_filtered, date, polygon)
        savi = utils.calculate_savi(s2_filtered, date, polygon)
        msavi = utils.calculate_savi(s2_filtered, date, polygon, 'M')

        # Create a dataframe row for the date
        df_acquisition= {'crop_field_name': field_name, 'acquisition_date': date, 'NDVI': ndvi,
                 'EOMI1': eomi1, 'EOMI2': eomi2, 'EOMI3': eomi3, 'EOMI4': eomi4, 'NBR2': nbr2,
                 'SAVI': savi, 'MSAVI': msavi, 'manure_dates': manure_dates}
        
        # Add row to the list
        acquisitions.append(df_acquisition)
    
    return acquisitions

In [10]:
# Create an empty list to store the data for each field
df_list = []

# Calculate all the indices for each field, for the selected time period
# In parallell, to improve performances (each core works on one single field)
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for index, field in fields_df.iterrows():
        futures.append(executor.submit(process_field, field))
    
    # Wait for all futures to complete and gather results
    for future in concurrent.futures.as_completed(futures):
        df_list.extend(future.result())

# Create a dataframe from the list of rows
fields_features_extracted_df = pd.DataFrame(df_list)

In [15]:
# Show the dataframe
fields_features_extracted_df

,crop_field_name,acquisition_date,NDVI,EOMI1,EOMI2,EOMI3,EOMI4,NBR2,SAVI,MSAVI,manure_dates
0,P-CLGT,2022-01-06,0.890879,-0.475203,0.478283,-0.266043,0.717336,0.364730,1.272070,0.942166,[2022-05-16]
1,P-CLGT,2022-01-16,0.890132,-0.435185,0.514562,-0.223194,0.740412,0.365419,1.271003,0.941776,[2022-05-16]
2,P-CLGT,2022-01-26,0.639473,-0.354393,0.169373,0.005410,0.364515,0.208003,0.913113,0.779930,[2022-05-16]
3,P-CLGT,2022-02-05,0.423970,-0.287130,0.049286,0.166670,0.156132,0.107899,0.605364,0.590960,[2022-05-16]
4,P-CLGT,2022-02-10,0.487526,-0.318114,0.032209,0.103522,0.199279,0.168300,0.696156,0.654858,[2022-05-16]
...,...,...,...,...,...,...,...,...,...,...,...
774,P-VG2,2022-10-01,0.587973,-0.303828,0.178242,0.067826,0.359690,0.193909,0.839567,0.740201,[2022-04-13]
775,P-VG2,2022-11-10,0.389072,-0.122577,0.198866,0.282058,0.316113,0.125169,0.555542,0.556166,[2022-04-13]
776,P-VG2,2022-11-12,0.393333,-0.133609,0.163594,0.268466,0.304470,0.148289,0.561634,0.562339,[2022-04-13]
777,P-VG2,2022-11-17,0.308817,-0.158042,0.078185,0.292526,0.177674,0.100908,0.440931,0.470664,[2022-04-13]


## Analysis

In [17]:
# General, simple yet useful statistics
fields_features_extracted_df.describe()

,NDVI,EOMI1,EOMI2,EOMI3,EOMI4,NBR2,SAVI,MSAVI
count,779.000000,779.000000,779.000000,779.000000,779.000000,779.000000,779.000000,779.000000
mean,0.444667,-0.161461,0.179976,0.221867,0.330907,0.169890,0.634937,0.585070
std,0.201061,0.141119,0.136863,0.174520,0.167721,0.065969,0.287091,0.206764
min,0.011629,-0.528310,-0.350105,-0.350925,-0.214569,0.007748,0.016606,0.022675
25%,0.304264,-0.275585,0.150263,0.097871,0.303373,0.139627,0.434457,0.462922
50%,0.458626,-0.159874,0.189342,0.241391,0.356751,0.174864,0.654873,0.625134
75%,0.584209,-0.054712,0.218365,0.374800,0.383015,0.198568,0.834198,0.736632
max,0.942142,0.160089,0.664984,0.530544,0.827342,0.376216,1.345279,0.970074


## Store the dataset containing all the extracted features, for all the fields

In [18]:
# Compressed .csv file, to take less memory space
filename = file_chooser.selected_path + "/" + file_chooser.selected_filename.split(".")[0] + "-features-extracted.gz"
fields_features_extracted_df.to_csv(filename, header=True, index=False, compression='gzip')